**Depósito: Somente valores positivos. Deve ser retornado em extrato

**Saques: Somente 3 por dia. Máximo de 500 por saque. Caso não tenha saldo o sistema exibe uma mensagem. Deve ser retornado os saques em extrato

**Extrato: Depósitos e Saques e no fim da listagem o saldo da conta. Deve-se usar o R$.

**Estabelecer um limite de 10 transações diárias para a conta 

**Se o usuário tentar fazer uma transação após atingir o limite, deve ser informado que ele excedeu o número de transações permitidas para aquele dia.

**Mostre no extrato, a data e hora de todas as transações



'''
Depósito: Somente valores positivos. Deve ser retornado em extrato
Saques: Somente 3 por dia. Máximo de 500 por saque. Caso não tenha saldo o sistema exibe uma mensagem. Deve ser retornado os saques em extrato
Extrato: Depósitos e Saques e no fim da listagem o saldo da conta. Deve-se usar o R$.

'''
         


In [ ]:
from sistema_bancario import *

while True:
        print(
            """
            ######MENU######

            Selecione uma Opção:
            [1] Criar Usuário
            [2] Criar Conta Corrente
            [3] Depositar
            [4] Sacar
            [5] Extrato
            [6] Sair

            """ 
        ) 

        option = int(input("Opção: "))

        if option == 6:
            print('Saindo...')
            break
        elif option == 1:
            name_user = input('Nome do Usuário: ')
            data_nasc = input('Data de Nascimento: Ex: 01/01/2000: ')
            cpf = input('CPF: Obs: somente números: ')
            logradouro = input('Logradouro: Ex: Rua das Acaúbas, 123, Vila Nova, São Paulo, SP, 12345-678 \n')
            dt_criacao = dt_operacao
            user = {'cpf': cpf,'nome': name_user, 'data_nasc':data_nasc, 'logradouro':logradouro}
            print(criar_usuario(user))
        elif option == 2:
             cpf = input('CPF: Obs: somente números: ')
             print(criar_conta_corrente(cpf))             
        elif option == 3:
            valor = float(input('Digite o valor para depósito: ')) 
            conta = input('Digite a conta: Ex: 1398-7: ')            
            deposito(valor, conta)
        elif option == 4:
            valor = float(input('Digite o valor para saque: ')) 
            conta = input('Digite a conta: Ex: 1398-7: ')
            print(saque(conta, valor))      
        elif option == 5:
            conta = input('Digite a conta: Ex: 1398-7: ')
            tamanho = 5
            result = extrato(conta)            
            # Imprime as 5 ultimas operações
            x = slice(5)
            for value in result['historico'][x]: # Imprimindo o extrato com os 5 ultimos movimentos
                print(f'{value["dt_operacao"]} - {value["valor"]} - {value["tipo"]}')
           
           
            print(f'O saldo total: R$ {result["saldo"]}')
             
        else:
            print('Opção inválida. Tente novamente.')                

